# 獲取ro跟q的CODE

In [ ]:
# 因為判決書系統擋webdriver，所以下列CODE無法使用

In [ ]:
'''
匯入套件
'''
# 操作 browser 的 API
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

# 整理 json 使用的工具
import json

# 執行 command 的時候用的
import os

# 引入 hashlib 模組
import hashlib

# 引入 regular expression 工具
import re

'''
Selenium with Python 中文翻譯文檔
參考網頁：https://selenium-python-zh.readthedocs.io/en/latest/index.html
selenium 啓動 Chrome 的進階配置參數
參考網址：https://stackoverflow.max-everyday.com/2019/12/selenium-chrome-options/
Mouse Hover Action in Selenium
參考網址：https://www.toolsqa.com/selenium-webdriver/mouse-hover-action/
'''
# headers
headers = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"

# 啟動瀏覽器工具的選項
options = webdriver.ChromeOptions()
# options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
options.add_argument("--start-maximized")         #最大化視窗
options.add_argument("--incognito")               #開啟無痕模式
options.add_argument("--disable-popup-blocking ") #禁用彈出攔截
options.add_argument("--disable-notifications")   #取消通知
options.add_argument("user-agent={}".format(headers)) #加headers

# 使用 Chrome 的 WebDriver (含 options)
driver = webdriver.Chrome( options = options )

url = 'https://law.judicial.gov.tw/FJUD/Default_AD.aspx'

# 整理所有取得資料的變數
listData = []

# 開始搜尋年份
SY=89
# 結束搜尋年份
EY=110

def visit():
    driver.get(url);
    sleep(1)
    
# 選案件類別
def setCategory():
    sleep(1)
    driver.find_element_by_id("vtype_M").click()
    sleep(2)    

# 填案由跟查詢區間及取得q的編碼以及該區間案件數
def search():
    # 填要查詢的內容跟時間區間
    for j in range(SY,EY+1):
        for i in range(1,12,2):
            sleep(1)
            txtInput = driver.find_element(By.CSS_SELECTOR,'input#jud_title')
            txtInput.send_keys('妨害名譽')
            txtInputSY = driver.find_element(By.CSS_SELECTOR,'input#dy1')
            txtInputSY.send_keys(j)
            txtInputSM = driver.find_element(By.CSS_SELECTOR,'input#dm1')
            txtInputSM.send_keys(i)
            txtInputSD = driver.find_element(By.CSS_SELECTOR,'input#dd1')
            txtInputSD.send_keys('1')
            txtInputEY = driver.find_element(By.CSS_SELECTOR,'input#dy2')
            txtInputEY.send_keys(j)
            txtInputEM = driver.find_element(By.CSS_SELECTOR,'input#dm2')
            txtInputEM.send_keys(i+1)
            txtInputED = driver.find_element(By.CSS_SELECTOR,'input#dd2')
            txtInputED.send_keys('31')
            sleep(1)
            # 進入網頁
            driver.find_element_by_id("btnQry").click()
            sleep(1)

            # 擷取查詢結果全部元素
            a_elms = driver.find_elements(By.CSS_SELECTOR, 'li.active > a')
            # 擷取連結元素
            aLink = a_elms[2].get_attribute('href')
            # 擷取文字元素
            aName = a_elms[2].get_attribute('text')
            # q會在連結字串的[64:96]這段位置
            qCode = aLink[64:96]
            # 文字元素擷取出來會是"查詢結果  123"，因此要正規化僅擷取數字部分
            regex = r'\d+'
            JugNum = re.search(regex, aName)[0]
            
            print(JugNum)
            print(qCode)
            listData.append({
                    "q": qCode,
                    "ro": JugNum,
                })
            sleep(1)
            # 擷取完成，跳回查詢頁
            driver.get(url)
            txtInput = driver.find_element(By.CSS_SELECTOR,'input#jud_title').clear()
            txtInputSY = driver.find_element(By.CSS_SELECTOR,'input#dy1').clear()
            txtInputSM = driver.find_element(By.CSS_SELECTOR,'input#dm1').clear()
            txtInputSD = driver.find_element(By.CSS_SELECTOR,'input#dd1').clear()
            txtInputEY = driver.find_element(By.CSS_SELECTOR,'input#dy2').clear()
            txtInputEM = driver.find_element(By.CSS_SELECTOR,'input#dm2').clear()
            txtInputED = driver.find_element(By.CSS_SELECTOR,'input#dd2').clear()
            sleep(1)
    # 印出結果，也可以改寫成輸出成csv檔
#     print(listData)
    
# 關閉瀏覽器
def close():
    driver.quit()
            
if __name__ == '__main__':
    visit()
    setCategory()
    search()
    close()

In [ ]:
listData
fp = open("ro_q_2m.json", "w", encoding='utf-8')
fp.write( json.dumps(listData, ensure_ascii=False) )
fp.close()

# 生成判決書網頁連結

In [ ]:
import pandas as pd
# 整理 json 使用的工具
import json
import html5lib
import lxml
import requests as req
from bs4 import BeautifulSoup as bs

In [ ]:
# 讀進ro跟q值
df = pd.read_csv('ro_q.csv')

# print first five row
print(df.head())

# 把q跟ro生成判決的連結
listLink = []
for i in df.index:
    for j in range(df["ro"][i]):
#         print("https://law.judicial.gov.tw/FJUD/data.aspx?ro={}&q={}&sort=DS&ot=in".format(j,df["q"][i]))
        listLink.append("https://law.judicial.gov.tw/FJUD/data.aspx?ro={}&q={}&sort=DS&ot=in".format(j,df["q"][i]))

In [ ]:
len(listLink)

In [ ]:
# 用pandas爬取網頁的table，然後直接寫成csv檔
for i in range(0,len(listLink)):
    Jug_text = pd.read_html(listLink[i])
    df_data = Jug_text[0]
    df_data.to_csv(f'./JudgeText/{i}.csv', index=False, header=False)

In [ ]:
from bs4 import BeautifulSoup
import requests as re
import sys
import traceback
import test

In [ ]:
# 建立放值的list       
num = []
date = []
tag = []
index = []

for i in range(0,len(listLink)):

    # 在一萬八千多筆的時候會停下, 設定exception
    try:
        url=listLink[i]
        
        response = re.get(url)
        
        soup = BeautifulSoup(response.text, "lxml")
        
        #print(soup.select('div.row div.col-td')[0].get_text())
        
        #num.append(soup.select('div.row div.col-td')[0].get_text().replace('\n', '').replace('\r', ''))
        # 用下面的strip 就能清除掉 字串頭尾的指定字元, 預設為換行符號與空格, 換行為\n(換行)+\r(回列最左邊)
        num.append(soup.select('div.row div.col-td')[0].get_text().strip())
        date.append(soup.select('div.row div.col-td')[1].get_text().strip())
        tag.append(soup.select('div.row div.col-td')[2].get_text().strip())
        index.append(i)
        
    except Exception as e:
        pass
    continue
# 建立dict已轉換 dataframe        
dict ={
        'index':index,
        "num":num,
        "date": date,
        "tag":tag
}

#print(num)

judgeinfo_df = pd.DataFrame(dict)

judgeinfo_df.to_csv("./judge_info.csv", header= True, index= True, encoding='UTF-8')

In [ ]:
judgeinfo_df.tail()

In [ ]:
judgeinfo_df = pd.DataFrame(dict)

judgeinfo_df.to_csv("./judge_info.csv", header= True, index= True, encoding='UTF-8')